In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt


In [2]:
customer_data = pd.read_csv("sales.csv")

print(customer_data.head())



   Unnamed: 0  store_ID  day_of_week        date  nb_customers_on_day  open  \
0      425390       366            4  2013-04-18                  517     1   
1      291687       394            6  2015-04-11                  694     1   
2      411278       807            4  2013-08-29                  970     1   
3      664714       802            2  2013-05-28                  473     1   
4      540835       726            4  2013-10-10                 1068     1   

   promotion state_holiday  school_holiday  sales  
0          0             0               0   4422  
1          0             0               0   8297  
2          1             0               0   9729  
3          1             0               0   6513  
4          1             0               0  10882  


In [3]:
# Visualize the data with correlation matrix
print("Shape of the data: ", customer_data.shape)
print("data types = ", customer_data.dtypes)
print(customer_data.describe())

Shape of the data:  (640840, 10)
data types =  Unnamed: 0              int64
store_ID                int64
day_of_week             int64
date                   object
nb_customers_on_day     int64
open                    int64
promotion               int64
state_holiday          object
school_holiday          int64
sales                   int64
dtype: object
          Unnamed: 0       store_ID    day_of_week  nb_customers_on_day  \
count  640840.000000  640840.000000  640840.000000        640840.000000   
mean   355990.675084     558.211348       4.000189           633.398577   
std    205536.290268     321.878521       1.996478           464.094416   
min         0.000000       1.000000       1.000000             0.000000   
25%    178075.750000     280.000000       2.000000           405.000000   
50%    355948.500000     558.000000       4.000000           609.000000   
75%    533959.250000     837.000000       6.000000           838.000000   
max    712044.000000    1115.000000    

In [4]:
missing_info = pd.DataFrame({
    "has_missing": customer_data.isna().any(),
    "missing_count": customer_data.isna().sum(),
    "missing_pct": (customer_data.isna().sum() / len(customer_data) * 100).round(2)
})
missing_info = missing_info[missing_info["has_missing"]]
print(missing_info)
cols_with_na = missing_info.index.tolist()


Empty DataFrame
Columns: [has_missing, missing_count, missing_pct]
Index: []


In [5]:
#Checking state-holiday values and conert to categorical
customer_data["state_holiday"].unique()
sh_dict = {'0': 0, 'a': 1, 'b': 2, 'c': 3}
customer_data["state_holiday"] = customer_data["state_holiday"].map(sh_dict)



In [6]:
customer_data["state_holiday"].unique()

array([0, 1, 3, 2], dtype=int64)

In [7]:
#1. Clean up and type-casting
#Drop the redundant index column
#The Unnamed: 0 column appears to be just the original row index. You can safely drop it:
customer_data = customer_data.drop(columns='Unnamed: 0')

#Convert date to datetime
#convert it so you can extract time-based features and do time‐series plots:
customer_data['date'] = pd.to_datetime(customer_data['date'])

In [8]:
# For second model approach : categorical approach
#Cast categorical fields
#Columns like day_of_week, open, promotion, state_holiday and school_holiday should be converted to category dtype in pandas. This both saves memory and makes plotting & grouping easier:

# for col in ['day_of_week','open','promotion','state_holiday','school_holiday']:
#    customer_data[col] = customer_data[col].astype('category')

In [9]:
#2. Feature engineering
#"Once your date is a datetime, you can create new features that often help boost model performance or reveal patterns in EDA:

#Temporal features

customer_data['year']   = customer_data['date'].dt.year
customer_data['month']  = customer_data['date'].dt.month
customer_data['dow']    = customer_data['date'].dt.dayofweek  # Monday=0 … Sunday=6
customer_data['weekofyear'] = customer_data['date'].dt.isocalendar().week

In [10]:
#Promotional lag features
#Capture whether a store was on promotion in the last 7 days, or count how many promo days in the past month.

customer_data = customer_data.sort_values(['store_ID','date'])
customer_data['promo_7d'] = (customer_data
    .groupby('store_ID')['promotion']
    .rolling(window=7, min_periods=1).sum()
    .reset_index(0,drop=True))
